In [32]:
# Detectar por recusividad el nombre, la ruta, el tamaño y la fecha de modificación de los archivos de un directorio

import os
import time

def listar_directorio(directorio, pdfs=None):
    if pdfs is None:
        pdfs = []
    
    # Recorremos todos los archivos en el directorio
    for nombre in os.listdir(directorio):
        ruta = os.path.join(directorio, nombre)
        if os.path.isdir(ruta):
            # Si es un directorio, llamamos recursivamente
            listar_directorio(ruta, pdfs)
        else:
            # Obtenemos información sobre el archivo
            info = os.stat(ruta)
            # Si el archivo es un PDF, lo añadimos a la lista
            if nombre.endswith('.pdf'):
                pdfs.append((nombre, ruta, info.st_size, time.ctime(info.st_mtime)))
                # NOMBRE RUTA SIZE FECHA
    return pdfs

pdfs = listar_directorio('../01_data/BBDD_Normativa_UPV')
pdfs[0]

('2013006700037300000387.pdf',
 '../01_data/BBDD_Normativa_UPV/2013/2013006700037300000387.pdf',
 550831,
 'Mon Nov 19 13:12:02 2018')

In [35]:
# Crear un código que haga una consulta a la base de datos para detectar si hay información de un archivo en la base de datos

import sqlite3

def conectar_bbdd(path):
    conexion = sqlite3.connect(path)
    cursor = conexion.cursor()
    return conexion, cursor

def desconectar_bbdd(conexion):
    conexion.commit()
    conexion.close()

def create_id(name):
    #crea un id (int) a partir del nombre del documento
    return hash(name)

def registrar_pdf(cursor, name, path, date, size):
    cursor.execute(f"""INSERT INTO pdfs VALUES ({create_id(name)}, '{name}', '{path}', {size}, '{date}')""")

def detectar_actualizaciones(cursor, name, path, size, date):
    # Detectamos si esta en la base de datos
    cursor.execute("""SELECT * FROM pdfs WHERE path = '{path}' """)
    datos = cursor.fetchall()
    if datos:
        # Vemos si la fecha y el tamaño coinciden o se han modificado
        if datos[0][2] != date or datos[0][3] != size:
            # Modificar BBDD
    
            # Registrar en la tabla de LOGs
            return "MOD"
        else:
            return "NOMOD"
    else:
        registrar_pdf(cursor, name, path, date, size)
        return "NEW"

In [36]:
conexion, cursor = conectar_bbdd('../01_data/project_database.db')

for pdf in pdfs:
    nombre, ruta, size, date = pdf
    detectar_actualizaciones(cursor, nombre, ruta, size, date)

In [37]:
desconectar_bbdd(conexion)